# A magnetostatic example

The magnetic field generated by a ferromagnet is an example where magnetostatics apply (since nothing is varying in time). 
A ferromagnet has all molecular [magnetic dipoles pointed in the same direction](https://en.wikipedia.org/wiki/Ferromagnetism)  which creates a nontrivial magnetization or *magnetic polarization* $\mathcal{M}$. Then  the magnetic flux density $\mathcal{B}$ is related to the magnetic field intensity $\mathcal{H}$ through 
$$
\mathcal B = \mu \mathcal H + \mathcal M.
$$
We will consider a bar magnet within which $\mathcal M$ can be modeled as a constant vector. Our discussion here is based on the [NGSolve tutorial 2.4](https://ngsolve.org/docu/nightly/i-tutorials/unit-2.4-Maxwell/Maxwell.html).

In [1]:
from math import pi
from netgen.csg import CSGeometry, OrthoBrick, Cylinder, unit_cube, Pnt
import ngsolve as ng
from ngsolve import x, y, z
from ngsolve import curl, grad, BilinearForm, LinearForm, dx
import ngsolve as ng
import netgen.gui
from ngsolve.internal import viewoptions, visoptions

Note the use of `-` and `*` on geometric objects.

In [2]:
box = OrthoBrick(Pnt(-3,-3,-3), Pnt(3,3,3)).bc("outer")
magnet = Cylinder(Pnt(-1,0,0), Pnt(1,0,0), 0.3) * \
         OrthoBrick(Pnt(-1,-3,-3), Pnt(1,3,3))
air = box - magnet

geo = CSGeometry()
geo.Add (air.mat("air"), transparent=True)
geo.Add (magnet.mat("magnet").maxh(1), col=(0.3,0.3,0.1))
geo.Draw()

In [3]:
mesh = ng.Mesh(geo.GenerateMesh(maxh=2, curvaturesafety=1))
mesh.Curve(3)

As discussed in class, one way to formulate the magnetostatics problem is as follows: find the magnetic flux density $\mathcal{B}$ as the curl of a vector potential $A$, i.e., $\mathcal{B} = \mathrm{curl }  A$ where $A$ solves the following problem after the introduction of a gauge condition through $\varphi$:
$$
\begin{aligned}
\mathrm{curl }( \mu^{-1} \mathrm{curl } A) + \mathrm{grad } \varphi 
& = \mathcal J + \mathrm{curl } (\mu^{-1}\mathcal M),
&& \text{ in } \Omega
\\
\mathrm{div } A & = 0 
&& \text{ in } \Omega
\\
n \times A & = 0 
&& \text{ on } \partial\Omega
\\
\varphi & = 0 
&& \text{ on } \partial\Omega.
\end{aligned}
$$
Since we have no imposed currents, for this problem $\mathcal J = 0$.  We shall use the lowest order Nedelec and Lagrange finite element spaces that we have studied so far.

In [4]:
mu0 = 4*pi*1e-7
mur = mesh.MaterialCF({'magnet': 1000}, default=1)
muinv = 1/(mu0*mur)
M = mesh.MaterialCF({'magnet': (1,0,0)}, default=(0,0,0))

In [5]:
N = ng.HCurl(mesh, order=0, type1=True, dirichlet='outer')
V = ng.H1(mesh, order=1, dirichlet='outer')
NV = ng.FESpace([N, V])

In [6]:
(A, p), (Z, q) = NV.TnT()

a = BilinearForm(NV)
a += muinv*curl(A)*curl(Z)*dx + grad(p)*Z*dx
a += grad(q)*A*dx 

f = LinearForm(NV)
f += muinv * M * curl(Z) * dx("magnet")

with ng.TaskManager():
    a.Assemble()
    f.Assemble()

In [7]:
Ap = ng.GridFunction(NV)
Ap.vec.data = a.mat.Inverse() * f.vec

In [8]:
B = curl(Ap.components[0])

viewoptions.clipping.enable=1
viewoptions.clipping.nx = 0 
viewoptions.clipping.ny = 0 
viewoptions.clipping.nz = -1
visoptions.clipsolution = 'vec'
ng.Draw(B, mesh, 'B', draw_surf=False)
